In [19]:
import pandas as pd
import numpy as np

df_all = pd.read_csv("cortical_metrics_1.3.csv")

# Exc and PV have sufficient number of cells, so we'll filter out non-V1 Exc and PV.
# SST and VIP are small populations, so let's keep also non-V1 neurons
exclude = (
    df_all["cell_type"].isnull()
    | df_all["cell_type"].str.contains("EXC")
    | df_all["cell_type"].str.contains("PV")
) & (df_all["ecephys_structure_acronym"] != "VISp")

df = df_all[~exclude]
print(f"Original: {df_all.shape[0]} cells,   filtered: {df.shape[0]} cells")

Original: 40294 cells,   filtered: 8799 cells


<ipython-input-19-017885074aaf>:4: DtypeWarning: Columns (60,62,65,67,69,72,119,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv("cortical_metrics_1.3.csv")


In [20]:
df

,ecephys_unit_id,L_ratio,age_in_days,amplitude_cutoff,anterior_posterior_ccf_coordinate,area_rf,azimuth_rf,c50_dg,cumulative_drift,d_prime,...,aic_diff1,aic_diff2,resp_sig,sig0.01,firing_rate_sp,preferred_angle,max_mean_rate(Hz),Ave_Rate(Hz),OSI,DSI
154,915960235,0.000023,100.0,0.500000,NaN,700.0,77.143,NaN,1043.19,5.309881,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,915960239,0.038601,100.0,0.500000,NaN,500.0,18.000,NaN,213.33,2.007172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,915960244,0.020035,100.0,0.468506,NaN,1000.0,36.000,NaN,195.20,2.828364,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,915960249,0.025151,100.0,0.309834,NaN,1100.0,58.182,NaN,369.14,2.877872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,915960253,0.014771,100.0,0.100638,NaN,1900.0,56.842,NaN,234.91,3.763859,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40289,951190691,0.026513,126.0,0.007937,7318.0,NaN,NaN,0.359831,27.31,4.376906,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40290,951190693,NaN,126.0,0.109200,7318.0,NaN,NaN,0.359831,0.00,3.829374,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40291,951190812,0.000002,126.0,0.240907,7318.0,NaN,NaN,0.359831,0.00,5.193001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40292,951190814,0.029493,126.0,0.450888,7318.0,NaN,NaN,0.359831,8.26,4.159769,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Count the number of observation from every session type
bo_counts = df[df["session_type"] == "brain_observatory_1.1"]["cell_type"].value_counts()
fc_counts = df[df["session_type"] == "functional_connectivity"]["cell_type"].value_counts()

# Some cells have very large values of RF. They are likely not-good fits, so ignore.
df.loc[(df["width_rf"] > 100), "width_rf"] = np.nan
df.loc[(df["height_rf"] > 100), "height_rf"] = np.nan

# define contrast response types
types = ["high", "band", "low"]
type_fracs = {}
type_fracs_err = {}
contrast_responsive = df["sig0.01"] == True
contrast_responsive_counts = df[contrast_responsive]["cell_type"].value_counts()
for t in types:
    type_counts = df[(df["best_model"] == t) & contrast_responsive]["cell_type"].value_counts()
    n = contrast_responsive_counts
    p = type_counts / n
    type_fracs[t] = p
    type_fracs_err[t] = np.sqrt(p * (1 - p) / n)

In [22]:
df.to_csv("V1_OSI_DSI_DF.csv", sep=" ", index=False)

In [ ]:
# set group statistics, and insert the cell counts for convenience
resource_tables = {}
resource_tables["median"] = df.groupby("cell_type").median()
resource_tables["mean"] = df.groupby("cell_type").mean()
resource_tables["sem"] = df.groupby("cell_type").sem()

for t in resource_tables.values():
    # insert counts and fractional elements that is common for all 3 tables
    t.insert(0, "n_cells_BO", bo_counts)
    t.insert(1, "n_cells_FC", fc_counts)
    t["n_cells_FC"][t["n_cells_FC"].isna()] = 0
    t["n_cells_FC"] = t["n_cells_FC"].astype(dtype=int)

    # t.insert(-1, "frac_highpass", type_fracs["high"])
    t["frac_highpass"] = type_fracs["high"]
    t["frac_bandpass"] = type_fracs["band"]
    t["frac_lowpass"] = type_fracs["low"]

# sem is a special case. re-insert the estimated error of the fraction
resource_tables["sem"]["frac_highpass"] = type_fracs_err["high"]
resource_tables["sem"]["frac_bandpass"] = type_fracs_err["band"]
resource_tables["sem"]["frac_lowpass"] = type_fracs_err["low"]

In [14]:
selected_metrics = [
    "n_cells_BO",
    "n_cells_FC",
    "g_osi_dg",
    "g_dsi_dg",
    "pref_sf_sg",
    "pref_tf_dg",
    "firing_rate_sp",
    "firing_rate_rf",
    "firing_rate_dg",
    "width_rf",
    "height_rf",
    "fano_dg",
    "frac_highpass",
    "frac_bandpass",
    "frac_lowpass",
]

# For the full list of metrics, refer to list(df.columns)
#
# See https://allensdk.readthedocs.io/en/latest/visual_coding_neuropixels.html
# for detailed descriptions of the metrics.


In [19]:
list(df.columns)

['ecephys_unit_id',
 'L_ratio',
 'age_in_days',
 'amplitude_cutoff',
 'anterior_posterior_ccf_coordinate',
 'area_rf',
 'azimuth_rf',
 'c50_dg',
 'cumulative_drift',
 'd_prime',
 'date_of_acquisition',
 'dorsal_ventral_ccf_coordinate',
 'ecephys_channel_id',
 'ecephys_probe_id',
 'ecephys_session_id',
 'ecephys_structure_acronym',
 'ecephys_structure_id',
 'elevation_rf',
 'f1_f0_dg',
 'fano_dg',
 'fano_dm',
 'fano_fl',
 'fano_ns',
 'fano_rf',
 'fano_sg',
 'firing_rate',
 'firing_rate_dg',
 'firing_rate_dm',
 'firing_rate_fl',
 'firing_rate_ns',
 'firing_rate_rf',
 'firing_rate_sg',
 'g_dsi_dg',
 'g_osi_dg',
 'g_osi_sg',
 'genotype',
 'has_lfp_data',
 'height_rf',
 'image_selectivity_ns',
 'isi_violations',
 'isolation_distance',
 'left_right_ccf_coordinate',
 'lfp_sampling_rate',
 'lifetime_sparseness_dg',
 'lifetime_sparseness_dm',
 'lifetime_sparseness_fl',
 'lifetime_sparseness_ns',
 'lifetime_sparseness_rf',
 'lifetime_sparseness_sg',
 'local_index',
 'max_drift',
 'mod_idx_dg',
 

In [15]:
pd.set_option("display.float_format", "{:0.2f}".format)
print('Median functional responses')
resource_tables["median"][selected_metrics]


Median functional responses


,n_cells_BO,n_cells_FC,g_osi_dg,g_dsi_dg,pref_sf_sg,pref_tf_dg,firing_rate_sp,firing_rate_rf,firing_rate_dg,width_rf,height_rf,fano_dg,frac_highpass,frac_bandpass,frac_lowpass
cell_type,,,,,,,,,,,,,,,
ALL_L1,50,91,0.18,0.11,0.04,2.00,0.83,0.00,0.40,25.90,28.65,3.37,0.83,0.17,NaN
EXC_L23,203,300,0.30,0.19,0.04,2.00,0.27,0.29,0.66,13.05,14.68,2.73,0.54,0.29,0.18
EXC_L4,301,328,0.20,0.12,0.04,2.00,1.06,1.01,2.05,25.28,28.29,3.31,0.61,0.30,0.09
EXC_L5,482,493,0.19,0.08,0.04,2.00,2.22,2.41,4.47,32.85,37.05,3.14,0.71,0.23,0.06
EXC_L6,191,153,0.42,0.08,0.04,2.00,0.93,1.31,2.71,31.96,34.75,3.34,0.76,0.18,0.06
PV_L23,75,58,0.11,0.09,0.08,2.00,3.10,3.38,5.20,17.98,21.95,6.20,0.67,0.17,0.17
PV_L4,98,81,0.11,0.07,0.04,2.00,3.89,4.34,7.72,28.14,28.83,6.56,0.62,0.22,0.16
PV_L5,99,101,0.12,0.06,0.08,2.00,4.58,4.78,10.25,31.78,34.51,6.59,0.79,0.15,0.06
PV_L6,56,34,0.16,0.06,0.06,2.00,4.61,5.27,11.86,46.37,48.53,5.59,0.81,0.15,0.04


In [5]:
print('Mean functional responses')
resource_tables["mean"][selected_metrics]


Mean functional responses


,n_cells_BO,n_cells_FC,g_osi_dg,g_dsi_dg,pref_sf_sg,pref_tf_dg,firing_rate_sp,firing_rate_rf,firing_rate_dg,width_rf,height_rf,fano_dg,frac_highpass,frac_bandpass,frac_lowpass
cell_type,,,,,,,,,,,,,,,
ALL_L1,50,91,0.33,0.26,0.06,2.34,2.93,0.77,1.34,2.35,16.26,4.28,0.83,0.17,NaN
EXC_L23,203,300,0.36,0.25,0.10,2.48,1.00,1.28,1.39,13.30,20.43,3.90,0.54,0.29,0.18
EXC_L4,301,328,0.26,0.17,0.11,2.83,2.52,2.75,3.52,26.94,30.80,55.90,0.61,0.30,0.09
EXC_L5,482,493,0.27,0.12,0.10,3.56,4.03,4.40,5.68,34.17,38.27,4.60,0.71,0.23,0.06
EXC_L6,191,153,0.45,0.13,0.10,3.39,2.33,2.62,3.52,34.15,33.82,5.05,0.76,0.18,0.06
PV_L23,75,58,0.13,0.11,0.10,2.02,4.93,5.40,6.57,21.90,25.46,8.25,0.67,0.17,0.17
PV_L4,98,81,0.13,0.09,0.08,2.47,6.38,7.07,9.27,29.37,32.41,8.76,0.62,0.22,0.16
PV_L5,99,101,0.14,0.07,0.10,3.13,9.22,9.38,14.02,33.41,38.48,10.53,0.79,0.15,0.06
PV_L6,56,34,0.18,0.07,0.10,3.31,8.99,9.20,14.36,46.73,46.02,7.62,0.81,0.15,0.04


In [6]:
print("SEM of functional responses")
resource_tables["sem"][selected_metrics]

SEM of functional responses


,n_cells_BO,n_cells_FC,g_osi_dg,g_dsi_dg,pref_sf_sg,pref_tf_dg,firing_rate_sp,firing_rate_rf,firing_rate_dg,width_rf,height_rf,fano_dg,frac_highpass,frac_bandpass,frac_lowpass
cell_type,,,,,,,,,,,,,,,
ALL_L1,50,91,0.04,0.04,0.01,0.20,0.89,0.17,0.19,26.06,14.63,0.39,0.15,0.15,NaN
EXC_L23,203,300,0.02,0.01,0.01,0.12,0.09,0.26,0.10,4.98,1.38,0.24,0.04,0.04,0.03
EXC_L4,301,328,0.01,0.01,0.01,0.13,0.16,0.17,0.17,1.96,1.09,51.61,0.04,0.03,0.02
EXC_L5,482,493,0.01,0.01,0.00,0.13,0.15,0.18,0.15,0.97,1.20,0.19,0.03,0.02,0.01
EXC_L6,191,153,0.02,0.01,0.01,0.22,0.18,0.19,0.16,2.23,3.27,0.28,0.05,0.04,0.03
PV_L23,75,58,0.01,0.01,0.01,0.20,0.44,0.50,0.43,1.31,1.67,0.58,0.08,0.06,0.06
PV_L4,98,81,0.01,0.01,0.01,0.19,0.54,1.02,0.48,1.53,1.66,0.66,0.07,0.06,0.05
PV_L5,99,101,0.01,0.01,0.01,0.27,0.86,0.90,0.84,1.76,2.00,0.88,0.05,0.04,0.03
PV_L6,56,34,0.02,0.01,0.01,0.44,1.20,1.24,1.26,3.40,4.16,0.67,0.08,0.07,0.04
